## 1. Read a csv file in memory

First, read the csv file with pandas (dataframe)

Plot a scatter for `JPM` (JP Morgan) vs `AXP` (American Express) to visually look at correlation.

In [0]:
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
fileurl = "https://s3-eu-west-1.amazonaws.com/training180529/data/djia_close.csv"

df = pd.read_csv(fileurl, sep=',',header=0)

In [0]:
plt.scatter(df[['AXP']], df[['JPM']])

#### Sort and index by date

In [0]:
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by= 'date').reset_index(drop=True)

In [0]:
df[['AXP', 'JPM', 'IBM']].plot()

#### Some data cleaning: identify columns with at least one NaN

In [0]:
df.columns[df.isna().any()].tolist()

## 2. Linear model

In [0]:
import tensorflow as tf

In [0]:
with tf.Session() as sess:
  x = tf.constant(df[['AXP']])
  y = tf.constant(df[['JPM']])
  
  weights = tf.Variable(tf.random_normal([1, 1], 0, .1, dtype=tf.float64))
  yhat = x @ weights
  
  yerror = tf.subtract(yhat, y)
  loss = tf.reduce_sum(tf.multiply(yerror, yerror))
  
  tf.global_variables_initializer().run()
  sess.run(loss)
  
  print(loss.eval())
  print(weights.eval())
  
  plt.plot(range(y.eval().shape[0]), y.eval(), 'g-')
  plt.plot(range(y.eval().shape[0]), yhat.eval(), 'b-')
  plt.plot(yerror.eval(), 'r-')
  

#### Manual guess of parameter: 

The average price of `JPM` divided by `AXP` is a good weight guess, here is a compute of mean ratio:

In [0]:
with tf.Session() as sess:
  x = tf.constant(df[['AXP']])
  y = tf.constant(df[['JPM']])
  
  x_m = tf.reduce_mean(y / x)
  print(x_m.eval())

#### grid search of optimal weight

In [0]:
losses = []
ws = []
gradients = []

with tf.Session() as sess:
  x = tf.constant(df[['AXP']])
  y = tf.constant(df[['JPM']])
  
  weights = tf.Variable(tf.random_normal([1, 1], 0, .1, dtype=tf.float64))
  yhat = x @ weights
  
  yerror = tf.subtract(yhat, y)
  loss = 0.5 * tf.reduce_sum(tf.multiply(yerror, yerror))
  gradient = tf.reduce_sum(tf.transpose(tf.multiply(x, yerror)), 1, keep_dims=True)
  
  update_weights = tf.assign(weights, weights + 0.1)
  
  tf.global_variables_initializer().run()
  
  for i in range(20):
    sess.run([loss, update_weights])
    losses.append(loss.eval())
    ws.append(weights.eval()[0])
    gradients.append(gradient.eval()[0])
  fig, ax1 = plt.subplots()
  color = 'tab:red'
  ax1.set_xlabel('weight')
  ax1.set_ylabel('Loss', color=color)
  ax1.plot(ws, losses, color=color)
  ax1.tick_params(axis='y', labelcolor=color)
  
  ax2 = ax1.twinx()
  color = 'tab:blue'
  ax2.set_ylabel('Gradient', color=color) 
  ax2.plot(ws, gradients, color=color)
  ax2.tick_params(axis='y', labelcolor=color)
  fig.tight_layout()

In [0]:
losses = []
ws = []
gradients = []

learning_rate = 1e-6

with tf.Session() as sess:
  x = tf.constant(df[['AXP']])
  y = tf.constant(df[['JPM']])
  
  weights = tf.Variable(tf.random_normal([1, 1], 0, .1, dtype=tf.float64))
  yhat = x @ weights
  
  yerror = tf.subtract(yhat, y)
  loss = 0.5 * tf.reduce_sum(tf.multiply(yerror, yerror))
  gradient = tf.reduce_sum(tf.transpose(tf.multiply(x, yerror)), 1, keep_dims=True)
  
  update_weights = tf.assign_sub(weights, learning_rate * gradient)
  
  tf.global_variables_initializer().run()
  
  for i in range(20):
    sess.run([loss, update_weights])
    
    losses.append(loss.eval())
    ws.append(weights.eval()[0])
    gradients.append(gradient.eval()[0])
    
    
  fig, axes = plt.subplots(1, 2)
  ax1 = axes[0]
  color = 'tab:red'
  ax1.set_xlabel('weight')
  ax1.set_ylabel('Loss', color=color)
  ax1.plot(ws, losses, color=color)
  ax1.tick_params(axis='y', labelcolor=color)
  
  ax2 = ax1.twinx()
  color = 'tab:blue'
  ax2.set_ylabel('Gradient', color=color) 
  ax2.plot(ws, gradients, color=color)
  ax2.tick_params(axis='y', labelcolor=color)
  
  axes[1].plot(losses)
  fig.tight_layout()
  
  print(weights.eval())
  print(loss.eval())
  

#### Use GradientDescentOptimizer

In [0]:
losses = []
ws = []

learning_rate = 1e-6

with tf.Session() as sess:
  x = tf.constant(df[['AXP']])
  y = tf.constant(df[['JPM']])
  
  weights = tf.Variable(tf.random_normal([1, 1], 0, .1, dtype=tf.float64))
  b = tf.Variable(tf.random_normal([1, 1], 0, .1, dtype=tf.float64))
  yhat = x @ weights + b
  
  yerror = tf.subtract(yhat, y)
  loss = tf.nn.l2_loss(yerror)
  
  update_weights = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  
  tf.global_variables_initializer().run()
  
  for i in range(20):
    sess.run([loss, update_weights])
    
    losses.append(loss.eval())
    ws.append(weights.eval()[0])
    gradients.append(gradient.eval()[0])
    
    
  fig, axes = plt.subplots(1, 2)
  ax1 = axes[0]
  color = 'tab:red'
  ax1.set_xlabel('weight')
  ax1.set_ylabel('Loss', color=color)
  ax1.plot(ws, losses, color=color)
  ax1.tick_params(axis='y', labelcolor=color)
  
  axes[1].plot(losses)
  fig.tight_layout()
  
  print(weights.eval())
  print(b.eval())
  print(loss.eval())
  